In [2]:
!pip install pyresparser fitz PyMuPDF pypdf2 >/dev/null

In [3]:
!ln -sf "NOUVELLE FLORE DU LIBAN ET DE LA SYRIE.pdf" document.pdf #alias

In [7]:
import fitz
document = 'document.pdf'
filename = document
search_term = "the"
pdf_document = fitz.open(filename)

In [8]:
doc = pdf_document
dir(doc)

['FontInfos',
 'FormFonts',
 'Graftmaps',
 'PDFCatalog',
 'PDFTrailer',
 'ShownPages',
 '__class__',
 '__contains__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__weakref__',
 '_addFormFont',
 '_delToC',
 '_delXmlMetadata',
 '_deleteObject',
 '_deletePage',
 '_do_links',
 '_dropOutline',
 '_embeddedFileAdd',
 '_embeddedFileDel',
 '_embeddedFileGet',
 '_embeddedFileIndex',
 '_embeddedFileInfo',
 '_embeddedFileNames',
 '_embeddedFileUpd',
 '_forget_page',
 '_getCharWidths',
 '_getMetadata',
 '_getNewXref',
 '_getOLRootNumber',
 '_getPDFfileid',
 '_getPDFroot',
 '_getPageInfo',
 '_getPageObjNumber',
 '_getPag

In [9]:
doc.getToC()

[[1, 'Abreviations', 77]]

In [10]:
doc.searchPageFor(528, 'Pteranthus')

[Rect(37.68000030517578, 99.17700958251953, 113.67733764648438, 113.85101318359375),
 Rect(37.91999816894531, 164.3159942626953, 94.519287109375, 179.55099487304688)]

In [11]:
# https://pymupdf.readthedocs.io/en/latest/faq/#how-to-analyze-font-characteristics

import fitz
import pprint, re

def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)


doc = fitz.open("document.pdf")
page = doc[528]

In [12]:
#header = 'Family	Genus	epithet	author	syn.	Plate	description	Floraison from	Floraison to	Floraison notes	L.	S.	Aire géogr.	Notes'
header = 'Family	GenusDescription	Genus	epithet	author	syn.	Plate	description	Floraison	L.	S.	Aire géogr.	Notes'
columns = header.split('	')
columns

['Family',
 'GenusDescription',
 'Genus',
 'epithet',
 'author',
 'syn.',
 'Plate',
 'description',
 'Floraison',
 'L.',
 'S.',
 'Aire géogr.',
 'Notes']

In [36]:
class Face:
    def __init__(self, font, size, color, flags=0):
        self.font = font
        self.size = size
        self.color = color
        self.flags = flags
        self.style = flags_decomposer(self.flags)
    def __str__(self):
        return f"Font: '{self.font}' ({self.style}), size {self.size:g}, color " + ("#%06x" % self.color)

def isfamily(t, fp):
    font = "Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000"
    if  re.match('^[A-Z]+$', t) or str(font) == fp:
        return True

    
#general Genus description
def isGenGenus(t, fp):
    font = "Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000"
    if re.match('[A-Z]+\s[A-Z][a-z]*\.\s\—\s[A-Z]([a-z]|\s|,|[a-zàâçéèêëîïôûùüÿñæœ .-])+', t) or str(font) == fp:
        #print(f'''Text: {t}\n{fp}\n\n''')
        return True
    
#continuation of general Genus description
def isGenGenusContinued(t, fp, prevText):
    #'Times-Roman' (serifed, proportional), size 11, color #000000
#     print("fon", fp)
#     print("text", t)
    font = "Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000"
    #re.match('([A-Z][a-z]|\s|,|[a-zàâçéèêëîïôûùüÿñæœ .-])+', t)  
#     print("prev",prevText)
    if font != str(fp) and prevText:
        #print(f'''Text: {t}\n{fp}\n\n''')
        return False
    return True

def prevHasPeriod(t):
    return re.match('.*(\.)$', t)
    
def isGenus(t, fp):
    font = "Font: 'Times-Bold' (serifed, proportional, bold), size 10.9449, color #000000"
    font2 = "Font: 'Times-Bold' (serifed, proportional, bold), size 11, color #000000"
    if (re.match('[A-Z][a-z]+', t) or re.match('[A-Z][a-z]+\s[a-z]+', t)) and (font == str(fp) or font2 == str(fp)):
        return True
    
def isEpithet(t, fp):
    font = "Font: 'Times-Bold' (serifed, proportional, bold), size 11, color #000000"
    font2 = "Font: 'Times-Bold' (serifed, proportional, bold), size 10.9449, color #000000"
    if re.match('\s[a-z]+', t) or (str(font) == fp or str(font2) == fp):
        #print(f'''Text: {t}\n{fp}\n\n''')
        return True
    
def isAuthor(t, fp):
    font = "Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000"
    if str(font) == fp:
        #print(f'''Text: {t}\n{fp}\n\n''')
        return True

data = {}


p = (doc[x] for x in range(516,604))
p = (doc[x] for x in range(527,604))
p

<generator object <genexpr> at 0x7fb82c9214d0>

In [37]:
#aaron was in the middle of working on this and it has an error we know about
# import re

# def floraison(doc, pagenum=0, block_start=0):
#     for page in next(p):
#         blocks = page.getText("dict", flags=11)["blocks"]
#         for x,b in enumerate(blocks[block_start:]):  # iterate through the text blocks
#             for y,l in enumerate(b["lines"]):  # iterate through the text lines
#                 for z,s in enumerate(l["spans"]):  # iterate through the text spans
#                     s["style"] = flags_decomposer(s["flags"])  # readable font flags
#                     face = Face(s["font"], s["size"], s["color"], s["flags"])
#                     if re.match("Floraison", s["text"]):
#                         yield (page.number, x, y, z, s["text"])

# p = (doc[x] for x in range(516,540))
# next(floraison(doc, x))

In [38]:
p = (doc[x] for x in range(527,604))
p

<generator object <genexpr> at 0x7fb82b60b250>

In [39]:
page.number

528

In [40]:
#             if isfamily(s["text"], face) and len(l["spans"]) == 1:
#                 item['Family'] = s["text"]
#                 print(x, len(blocks), y, z, s["text"], face)
#                 break
#             if not item['Family']:
#                 break
#             if s['font'] == 'Times-Bold':
#                 item['Genus'], *item['epithet'] = s["text"].split(' ')
#                 item['epithet'] = item['epithet'][0] if item['epithet'] else l["spans"][1]["text"]
#                 currsize = s['size']
#                 print(x, len(blocks), y, len(b["lines"]), z, s["text"], face)
#                 from pprint import pprint
#                 print(x, len(blocks), y, z, pprint(item))
#                 print(); print(); 
#                 break
#             elif item['Genus'] and (not item['author']) and (s['font'] == 'Times-Roman'):
#                 item['author'] = s["text"]
#                 print(x, len(blocks), y, len(b["lines"]), z, s["text"], face)
#                 print(item)
#                 break

In [41]:
import pandas as pd

In [42]:
frame = pd.DataFrame(columns = ['text', 'font', 'size', 'style', 'color', 'flags'])

In [43]:
item = dict.fromkeys(columns)

In [44]:
#simplify
item = dict.fromkeys(columns)
p = (doc[x] for x in range(528,604))
page = next(p)
# read page text as a dictionary, suppressing extra spaces in CJK fonts
blocks = page.getText("dict", flags=11)["blocks"]
onGenGenus = False
onGenus = False
for x,b in enumerate(blocks): # iterate through the text blocks
    for y,l in enumerate(b["lines"]):  # iterate through the text lines
        for z,s in enumerate(l["spans"]):  # iterate through the text spans
            s["style"] = flags_decomposer(s["flags"])  # readable font flags
            face = Face(s["font"], s["size"], s["color"], s["flags"])
            #creating a data frame for analyzed
#             values = [s['text'],s['font'], s['size'], s['style'], s['color'], s['flags']]
#             columns = ['text', 'font', 'size', 'style', 'color', 'flags']
#             zipped = zip(columns, values)
#             a_dictionary = dict(zipped)
#             frame = frame.append(a_dictionary,ignore_index=True)
#             print(frame)
#             print(s['text'], "***",s['font'], "***", s['size'], "***", s['style'], "***", s['color'], "***", s['flags'],"********************************************************\n")
            
            if onGenGenus:
                
                #re.match('.*(\.)$', s['text'])
                if isGenGenusContinued(s["text"], face, prevPeriod):
                    item['GenusDescription'] += s["text"]
                    prevPeriod = prevHasPeriod(s["text"])
                    item['GenusDescription'] += s["text"]
#                     print("description")
#                     print(x, len(blocks), y, z, s["text"], face)
                else:
                    onGenGenus = False
#                 if z + 1 < len(l["spans"]) and l['spans'][z+1]['font'] == 'Times-Bold':
#                     #write more
#                     print(x, len(blocks), y, z, s["text"], face)

#                 print("description")
#                 print(item['GenusDescription'])

            if isfamily(s["text"], face) and len(l["spans"]) == 1:
                item['Family'] = s["text"]
                print("family")
                print(x, len(blocks), y, z, s["text"], face)
#             elif isfamily(s["text"], face) and len(l["spans"]) == 1:
#                 item['Family'] = s["text"]
#                 print(x, len(blocks), y, z, s["text"], face)
#                 break

            elif isGenGenus(s["text"], face):
                item['GenusDescription'] = s["text"]
                onGenGenus = True
                if not re.match('.*(\.)$', s['text']):
                    prevPeriod = False
                else:
                    prevPeriod = True
            elif not item['Genus'] and isGenus(s["text"], face):
                if re.match(".*\s.*", s["text"]):
                    item['Genus'] = s["text"].split()[0]
                    item['epithet'] = s["text"].split()[1]
                else:
                    item['Genus'] = s["text"]
                    onGenus = True
                print(s["text"])
            elif onGenus:
                item['epithet'] = s["text"]
                onGenus = False
#             elif s['font'] == 'Times-Bold' and not item['Genus'] and not item['epithet'] and re.match("[A-Z].+\s.+", s["text"]):
#                 item['Genus'], *item['epithet'] = s["text"].split(' ')
#                 item['epithet'] = item['epithet'][0] if item['epithet'] else l["spans"][1]["text"]
#                 currsize = s['size']
# #                print(x, len(blocks), y, z, s["text"], font_properties)
#                 print("both")
#                 print(x, len(blocks), y, len(b["lines"]), z, s["text"], face)
#             elif isGenus(s["text"], face) and not item['Genus'] and re.match("[A-Z].+", s["text"]):
#                 item['Genus'] = s["text"]
#                 print("genus")
#                 print(x, len(blocks), y, z, s["text"], face)
#             elif isEpithet(s["text"], face) and item['Genus'] and not item['epithet'] and re.match("\s.*", s["text"]):
#                 item['epithet'] = s["text"]
#                 print("epithet")
#                 print(x, len(blocks), y, z, s["text"], face)
print(item)

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Pteranthus
{'Family': None, 'GenusDescription': 'SCLERANTHUS L. — Herbes de petite taille, à feuilles linéaires-sétacées.Calice à tube urcéolé, à la fin durci. 5 sépales mutiques. Pétales filiformes, parfoisCalice à tube urcéolé, à la fin durci. 5 sépales mutiques. Pétales filiformes, parfoisavortés. Étamines 5. Ovaire libre. Styles 2, distincts.avortés. Étamines 5. Ovaire libre. Styles 2, distincts.', 'Genus': 'Pteranthus', 'epithet': ' dichotomus', 'author': None, 'syn.': None, 'Plate': None, 'description': None, 'Floraison': None, 'L.': None, 'S.': None, 'Aire géogr.': None, 'Notes': None}


In [61]:
#print info
arr2 = []

# read page text as a dictionary, suppressing extra spaces in CJK fonts

for page in (doc[x] for x in range(528,604)):
    blocks = page.getText("dict", flags=11)["blocks"]
    onGenGenus = False
    onGenus = False
    for x,b in enumerate(blocks): # iterate through the text blocks
        for y,l in enumerate(b["lines"]):  # iterate through the text lines
            for z,s in enumerate(l["spans"]):  # iterate through the text spans
                s["style"] = flags_decomposer(s["flags"])  # readable font flags
                face = Face(s["font"], s["size"], s["color"], s["flags"])
                arr2.append([s["text"], Face(s["font"], s["size"], s["color"], s["flags"]), page])
                #creating a data frame for analyze


In [62]:
block_frame = pd.DataFrame(arr2)

In [63]:
block_frame.to_csv("blocks4.csv")

In [101]:
import numpy as np

In [123]:
frame = pd.DataFrame(columns = columns)

In [124]:
frame.loc[len(frame)] = 0

In [125]:
frame

,Family,GenusDescription,Genus,epithet,author,syn.,Plate,description,Floraison,L.,S.,Aire géogr.,Notes
0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [126]:
frame.loc[len(frame)] = 0

In [137]:
#genus description
frame = pd.DataFrame(columns = columns)
arr = []
for page in (doc[x] for x in range(450,605)):
    # page = next(p)
    # read page text as a dictionary, suppressing extra spaces in CJK fonts
    blocks = page.getText("dict", flags=11)["blocks"]
    onGenGenus = False
    for x,b in enumerate(blocks): # iterate through the text blocks
        for y,l in enumerate(b["lines"]):  # iterate through the text lines
            for z,s in enumerate(l["spans"]):  # iterate through the text spans
                s["style"] = flags_decomposer(s["flags"])  # readable font flags
                face = Face(s["font"], s["size"], s["color"], s["flags"])
                #creating a data frame for analyzed
    #             values = [s['text'],s['font'], s['size'], s['style'], s['color'], s['flags']]
    #             columns = ['text', 'font', 'size', 'style', 'color', 'flags']
    #             zipped = zip(columns, values)
    #             a_dictionary = dict(zipped)
    #             frame = frame.append(a_dictionary,ignore_index=True)
    #             print(frame)
    #             print(s['text'], "***",s['font'], "***", s['size'], "***", s['style'], "***", s['color'], "***", s['flags'],"********************************************************\n")

                if onGenGenus:

                    #re.match('.*(\.)$', s['text'])
                    if isGenGenusContinued(s["text"], face, prevPeriod):
                        arr[-1][0] += s["text"]
                        frame.iloc[-1]['GenusDescription'] += s["text"]
                        item['GenusDescription'] += s["text"]
                        prevPeriod = prevHasPeriod(s["text"])
                        item['GenusDescription'] += s["text"]
                        print("description")
                        print(x, len(blocks), y, z, s["text"], face)
                    else:
                        onGenGenus = False
    #                 if z + 1 < len(l["spans"]) and l['spans'][z+1]['font'] == 'Times-Bold':
    #                     #write more
    #                     print(x, len(blocks), y, z, s["text"], face)


                if isfamily(s["text"], face) and len(l["spans"]) == 1:
                    item['Family'] = s["text"]
                    print("family")
                    print(x, len(blocks), y, z, s["text"], face)
    #             elif isfamily(s["text"], face) and len(l["spans"]) == 1:
    #                 item['Family'] = s["text"]
    #                 print(x, len(blocks), y, z, s["text"], face)
    #                 break

                elif isGenGenus(s["text"], face):
                    item['GenusDescription'] = s["text"]
                    onGenGenus = True
                    if not re.match('.*(\.)$', s['text']):
                        prevPeriod = False
                    else:
                        prevPeriod = True
                    print("description")
                    print(x, len(blocks), y, z, s["text"], face)
                    arr.append([s['text'], page])
                    frame.loc[len(frame)] = 0
                    frame.iloc[-1]['GenusDescription'] = s["text"]
                elif isGenus(s["text"], face):
                    frame.loc[len(frame)] = 0
                    if re.match(".*\s.*", s["text"]):
                        arr.append(["genus", s["text"].split()[0], s["text"].split()[1], page])
                        frame.iloc[-1]['Genus'] = s["text"].split()[0]
                        frame.iloc[-1]['epithet'] = s["text"].split()[1]
                    else:
                        arr.append(["genus", s["text"].split()[0], 0, page])
                        frame.iloc[-1]['Genus'] = s["text"].split()[0]
                        onGenus = True
                    print(s["text"])
                elif onGenus:
                    arr[-1][2] = s["text"]
                    frame.iloc[-1]['epithet'] = s["text"]
                    onGenus = False
# print(item)
# print(arr)
frame

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Urtica membranacea
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Urtica pilulifera
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Urtica dioica
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Urtica fragilis
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Parietaria judaica
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Arceuthobium
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
family
14 22 0 0 ARISTOLOCHIACEAE Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000
hi
hi
hi
hi
hi
hi
hi
hi
description
17 22 0 0 ARISTOLOCHIA L. — Herbes vivaces ou lianes, rarement arbrisseaux. Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000
description
18 22 0 0 Feuilles parfois entières, parfois lobées, le plus souvent cordées à la base, l'échancrure Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000
hi
description
18 22 1 0 dessinant un sinus plus ou moins ouvert, entre deux lobes parfois plus longs que Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000
hi
description
18 22 2 0 larges. Pédoncule

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Polygonum cognatum
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Polygonum setosum
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Polygonum libani
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
h

Atriplex micrantha
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Atriplex rosea
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Atriplex
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Atriplex
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Atriplex hastata
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Atriplex patula
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi


Salsola jordanicola
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Salsola incanescens
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Salsola autrani
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Salsola crassa
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Salsola subcrassa
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Salsola zenobiae
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Salsola tetrandra
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi


...

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Minuartia decipiens
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Minuartia montana
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Minuartia mesogitana
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Minuartia
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
h

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Agrostemma githago
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Melandrium
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Melandrium eriocalycinum
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi


hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Vaccaria oxyodonta
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
description
10 28 0 0 SAPONARIA L. — Plantes annuelles ou vivaces, à calice cylindrique ou Font: 'ETVWNV+Times-Roman' (serifed, proportional), size 10.4025, color #000000
description
11 28 0 0 oblong-cylindrique, parcouru par des nervures nombreuses généralement non sail- Font: 'ETVWNV+Times-Roman' (serifed, proportional), size 10.4025, color #000000
hi
description
11 28 1 0 lantes et peu visibles. Pétales 5, à onglet étroit bilamellé dans sa longueur. Lame Font: 'ETVWNV+Times-Roman' (serifed, proportional), size 10.4025, color #000000
hi
description
11 28 2 0 souvent bien distincte de l'onglet, nue Font: 'ETVW

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
h

,Family,GenusDescription,Genus,epithet,author,syn.,Plate,description,Floraison,L.,S.,Aire géogr.,Notes
0,0,0,Urtica,membranacea,0,0,0,0,0,0,0,0,0
1,0,0,Urtica,pilulifera,0,0,0,0,0,0,0,0,0
2,0,0,Urtica,dioica,0,0,0,0,0,0,0,0,0
3,0,0,Urtica,fragilis,0,0,0,0,0,0,0,0,0
4,0,0,Parietaria,judaica,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0,0,Dianthus,micranthus,0,0,0,0,0,0,0,0,0
345,0,0,Dianthus,pachypetalus,0,0,0,0,0,0,0,0,0
346,0,0,Dianthus,libanotis,0,0,0,0,0,0,0,0,0
347,0,0,Dianthus,crinitus,0,0,0,0,0,0,0,0,0


In [138]:
frame.tail()

,Family,GenusDescription,Genus,epithet,author,syn.,Plate,description,Floraison,L.,S.,Aire géogr.,Notes
344,0,0,Dianthus,micranthus,0,0,0,0,0,0,0,0,0
345,0,0,Dianthus,pachypetalus,0,0,0,0,0,0,0,0,0
346,0,0,Dianthus,libanotis,0,0,0,0,0,0,0,0,0
347,0,0,Dianthus,crinitus,0,0,0,0,0,0,0,0,0
348,0,0,Dianthus,orientalis,0,0,0,0,0,0,0,0,0


In [136]:
frame.to_csv("test.csv")

In [79]:
pd.DataFrame(arr).head()

,0,1,2,3
0,genus,Urtica,membranacea,page 450 of document.pdf
1,genus,Urtica,pilulifera,page 450 of document.pdf
2,genus,Urtica,dioica,page 450 of document.pdf
3,genus,Urtica,fragilis,page 451 of document.pdf
4,genus,Parietaria,judaica,page 452 of document.pdf


In [94]:
genus_descrip_frame = pd.DataFrame(arr, columns = ["a", "b", "c", "d"])

In [95]:
genus_descrip_frame

,a,b,c,d
0,genus,Urtica,membranacea,page 450 of document.pdf
1,genus,Urtica,pilulifera,page 450 of document.pdf
2,genus,Urtica,dioica,page 450 of document.pdf
3,genus,Urtica,fragilis,page 451 of document.pdf
4,genus,Parietaria,judaica,page 452 of document.pdf
...,...,...,...,...
344,genus,Dianthus,micranthus,page 603 of document.pdf
345,genus,Dianthus,pachypetalus,page 603 of document.pdf
346,genus,Dianthus,libanotis,page 604 of document.pdf
347,genus,Dianthus,crinitus,page 604 of document.pdf


In [98]:
genus_descrip_frame.iloc[-1]["a"] = "hi"

In [99]:
genus_descrip_frame.iloc[[-1]]

,a,b,c,d
348,hi,Dianthus,orientalis,page 604 of document.pdf


In [81]:
genus_descrip_frame.to_csv("genus5.csv")

In [237]:
frame

,text,font,size,style,color,flags


In [221]:
frame.to_excel('analyzed.xlsx', index = False, header=True)

In [222]:
"""
print(s['text'], "***",s['font'], "***", s['size'], "***", s['style'], "***", s['color'], "***", s['flags'],"********************************************************")


#family
CARYOPHYLLACEAE *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

15 18 0 0 CARYOPHYLLACEAE Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000
Plantes herbacées ou sous-frutescentes. Feuilles simples, opposées, décussées, *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

très rarement alternes. Fleurs habituellement en cymes bipares, actinomorphes, le *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

plus souvent hermaphrodites, parfois polygames, plus rarement dioïques. Périanthe *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

simple ou double, les pétales étant alors d'origine staminale. Calice libre ou *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

 gamo- *** Times-Roman *** 10.91801929473877 *** serifed, proportional *** 0 *** 4 ********************************************************

phylle. Corolle souvent onguiculée, toujours libre, et complétée parfois par des *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

ligules, constituant une couronne. Fruit en capsule s'ouvrant habituellement au *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

sommet, exceptionnellement une baie.  *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

— *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************



#other info
NOUVELLE FLORE  *** Times-Roman *** 8.954887390136719 *** serifed, proportional *** 0 *** 4 ********************************************************

451 *** Times-Roman *** 8.954887390136719 *** serifed, proportional *** 0 *** 4 ********************************************************

Sous-famille des *** Times-Italic *** 10.94486141204834 *** italic, serifed, proportional *** 0 *** 6 ********************************************************

 PARONYCHIOIDEAE *** Times-Italic *** 11.0 *** italic, serifed, proportional *** 0 *** 6 ********************************************************

Feuilles habituellement stipulées. Sépales libres ou faiblement concrescents. *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

Corolle le plus souvent absente. *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

PTERANTHUS Forsk. — Herbe annuelle, charnue, à feuilles opposées, fas- *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

ciculifères, ce qui les rend pseudo-verticillées. Cymes *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

 biflores *** Times-Roman *** 11.0 *** serifed, proportional *** 0 *** 4 ********************************************************

 portés par un pédon- *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

cule commun accrescent, large, oblong, aplati et tombant avec lui. Fleurs sessiles, *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

latérales. Calice 4-partite à sépales oblongs-linéaires, prolongés par un appendice *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

vertical scarieux-marginé. 4 étamines. Pas de corolle. *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************






#important info
Pteranthus *** Times-Bold *** 10.94486141204834 *** serifed, proportional, bold *** 0 *** 20 ********************************************************

 dichotomus *** Times-Bold *** 11.0 *** serifed, proportional, bold *** 0 *** 20 ********************************************************

 Forsk. — *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

 P. echinatus *** Times-Italic *** 10.94486141204834 *** italic, serifed, proportional *** 0 *** 6 ********************************************************

 Desf. *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

 (PI. *** Times-Roman *** 10.891444206237793 *** serifed, proportional *** 0 *** 4 ********************************************************

 CLV, n. 1). — *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

0. Tige très rameuse à la base, couchée ou ascendante, 10-40 cm. triangulaires, *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

plus ou moins dentées, opposées et fasciculifères, les feuilles des fascicules plus courtes *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

et plus fines. Pédoncules floraux aux aisselles des feuilles supérieures, papilleux, *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

hirsutes, d'abord cylindriques, mais se développant rapidement en une sorte de *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

raquette ovale, atténuée à la base. Fleurs sessiles, à 4 sépales, prolongés par un *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

appendice scarieux, lui-même terminé par une courte épine, et entourées de rameaux *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

stériles armés de plusieurs épines, les égalant sans les dépasser. *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************

Floraison: mars-mai. Régions arides. *** Times-Roman *** 8.954887390136719 *** serifed, proportional *** 0 *** 4 ********************************************************

S. *** Times-Roman *** 8.954887390136719 *** serifed, proportional *** 0 *** 4 ********************************************************

  *** Times-Italic *** 8.954887390136719 *** italic, serifed, proportional *** 0 *** 6 ********************************************************

St. *** Times-Italic *** 8.954887390136719 *** italic, serifed, proportional *** 0 *** 6 ********************************************************

 Palmyre *** Times-Roman *** 9.0 *** serifed, proportional *** 0 *** 4 ********************************************************

 (Sam, Mt, Pb), Palmyre à Arak, *** Times-Roman *** 8.954887390136719 *** serifed, proportional *** 0 *** 4 ********************************************************

 Bir-al-'Alaniyé *** Times-Roman *** 9.0 *** serifed, proportional *** 0 *** 4 ********************************************************

 (Pb), *** Times-Roman *** 8.954887390136719 *** serifed, proportional *** 0 *** 4 ********************************************************

 Jabal-et-Tenf *** Times-Roman *** 9.0 *** serifed, proportional *** 0 *** 4 ********************************************************

 (Bb). *** Times-Roman *** 8.954887390136719 *** serifed, proportional *** 0 *** 4 ********************************************************

Sud. *** Times-Italic *** 8.954887390136719 *** italic, serifed, proportional *** 0 *** 6 ********************************************************

 Hammé *** Times-Roman *** 9.0 *** serifed, proportional *** 0 *** 4 ********************************************************

 (Pb). *** Times-Roman *** 8.954887390136719 *** serifed, proportional *** 0 *** 4 ********************************************************

Aire géogr. —• Afrique du Nord, *** Times-Roman *** 8.954887390136719 *** serifed, proportional *** 0 *** 4 ********************************************************

 Egypte, *** Times-Roman *** 8.953801155090332 *** serifed, proportional *** 0 *** 4 ********************************************************

 Ethiopie, *** Times-Roman *** 8.961397171020508 *** serifed, proportional *** 0 *** 4 ********************************************************

 Chypre, Palestine sud, *** Times-Roman *** 8.954887390136719 *** serifed, proportional *** 0 *** 4 ********************************************************

 TransJordanie, *** Times-Roman *** 8.976929664611816 *** serifed, proportional *** 0 *** 4 ********************************************************

 Syrie, *** Times-Roman *** 8.954887390136719 *** serifed, proportional *** 0 *** 4 ********************************************************

Iraq, Iran, Géorgie. *** Times-Roman *** 8.954887390136719 *** serifed, proportional *** 0 *** 4 ********************************************************
"""

'\nprint(s[\'text\'], "***",s[\'font\'], "***", s[\'size\'], "***", s[\'style\'], "***", s[\'color\'], "***", s[\'flags\'],"********************************************************")\n\n\n#family\nCARYOPHYLLACEAE *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************\n\n15 18 0 0 CARYOPHYLLACEAE Font: \'Times-Roman\' (serifed, proportional), size 10.9449, color #000000\nPlantes herbacées ou sous-frutescentes. Feuilles simples, opposées, décussées, *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************\n\ntrès rarement alternes. Fleurs habituellement en cymes bipares, actinomorphes, le *** Times-Roman *** 10.94486141204834 *** serifed, proportional *** 0 *** 4 ********************************************************\n\nplus souvent hermaphrodites, parfois polygames, plus rarement dioïques. Périanthe *** Times-Roman *** 10.94486141204

In [ ]:
col1 = {"CARYOPHYLLACEAE", "Times-Roman", "10.94486141204834", "serifed, proportional", "0", "4"}
col2 = {"Pteranthus", "Times-Bold", "10.94486141204834", "serifed, proportional, bold", "0", "20"}
col3 = {"dichotomus", "Times-Bold", "11.0", "serifed, proportional, bold", "0", "20"}
col4 = {"Forsk.", "Times-Roman", "10.94486141204834", "serifed, proportional", "0", "4"}
col5 = {"P. echinatus", "Times-Italic", "10.94486141204834", "italic, serifed, proportional", "0", "6"}
col6 = {"Desf.", "Times-Roman", "10.94486141204834", "serifed, proportional", "0", "4"}
col7 = {"(PI.", "Times-Roman", "10.891444206237793", "serifed, proportional"}

In [42]:
# #simplify
# page = next(p)
# # read page text as a dictionary, suppressing extra spaces in CJK fonts
# blocks = page.getText("dict", flags=11)["blocks"]
# for x,b in enumerate(blocks):  # iterate through the text blocks
#     for y,l in enumerate(b["lines"]):  # iterate through the text lines
#         for z,s in enumerate(l["spans"]):  # iterate through the text spans
#             s["style"] = flags_decomposer(s["flags"])  # readable font flags
#             face = Face(s["font"], s["size"], s["color"], s["flags"])
#             print(s['text'], "***********************************************************")
#             if isfamily(s["text"], face) and len(l["spans"]) == 1:
#                 item['Family'] = s["text"]
#                 print(x, len(blocks), y, z, s["text"], face)
#                 break
#             if not item['Family']:
#                 break
#             if s['font'] == 'Times-Bold':
#                 item['Genus'], *item['epithet'] = s["text"].split(' ')
#                 item['epithet'] = item['epithet'][0] if item['epithet'] else l["spans"][1]["text"]
#                 currsize = s['size']
# #                print(x, len(blocks), y, z, s["text"], font_properties)
#                 print(x, len(blocks), y, len(b["lines"]), z, s["text"], face)
# #                 nn, xx, yy, zz, tt = next(floraison(page, x))
# #                 item['Floraison'] = tt
# #                 for q in range(x, max(x+15, len(blocks))):
# #                     for ll in blocks[q]["lines"]:
# #                         s = ll["spans"][0]
# #                         text, *rest = s['text'].split(' ')
# #                         if text == "Floraison:":
# #                             face = Face(s["font"], s["size"], s["color"], s["flags"])
# #                             print(face, s['text'])
# #                             for ss in ll["spans"]:
# #                                 #print(ss['text'], end=" ")
# #                                 pass
#                 from pprint import pprint
#                 print(x, len(blocks), y, z, pprint(item))
#                 #print([a["text"] for a in l["spans"]])
#                 print(); print(); 
#                 break
#             elif item['Genus'] and (not item['author']) and (s['font'] == 'Times-Roman'):
#                 item['author'] = s["text"]
#                 print(x, len(blocks), y, len(b["lines"]), z, s["text"], face)
#                 print(item)
#                 break

NOUVELLE FLORE  ***********************************************************
451 ***********************************************************
Sous-famille des ***********************************************************
 PARONYCHIOIDEAE ***********************************************************
Feuilles habituellement stipulées. Sépales libres ou faiblement concrescents. ***********************************************************
Corolle le plus souvent absente. ***********************************************************
PTERANTHUS Forsk. — Herbe annuelle, charnue, à feuilles opposées, fas- ***********************************************************
ciculifères, ce qui les rend pseudo-verticillées. Cymes ***********************************************************
 biflores ***********************************************************
 portés par un pédon- ***********************************************************
cule commun accrescent, large, oblong, aplati et tombant avec lui. Fleurs se

In [17]:
search = ((x, doc.searchPageFor(x, "Floraison", 100, False)) for x in range(516,520))

In [18]:
next(search)

(516,
 [Rect(42.0, 69.48302459716797, 77.89947509765625, 81.48902130126953),
  Rect(42.0, 324.6029968261719, 78.0272445678711, 336.6089782714844)])

In [19]:
p = (doc[x] for x in range(516,520))

In [20]:
page = next(p)
rects = page.searchFor("Floraison")
rects[0]

Rect(42.0, 69.48302459716797, 77.89947509765625, 81.48902130126953)

In [21]:
page.getTextBlocks()

[(177.83999633789062,
  18.363027572631836,
  409.36810302734375,
  30.36902618408203,
  'NOUVELLE FLORE  439',
  0,
  0),
 (24.0,
  40.73700714111328,
  409.758544921875,
  67.1710205078125,
  'centes. Sépales linéaires, spatules, denticulés, lacérés. Staminodes plus courts que\nleur partie soudée. Fruit un peu accrescent, à la fin glabre.',
  1,
  0),
 (42.0,
  69.48302459716797,
  409.81085205078125,
  94.92902374267578,
  'Floraison: automne.\nS. St. Entre Alep et Bagdad (Oliv), Palmyre (Mt, Zoh, Pb), Deir-ez-Zor (Gb), entre Mey-',
  2,
  0),
 (23.760000228881836,
  92.52300262451172,
  409.95843505859375,
  114.12897491455078,
  'yadine et Salihiyé (Hd-Mz), W. de Deir-ez-Zor, entre Abou Kemal et Meyyadine (Pb), entre\nSoukhné et Deir-ez-Zor (Dlb).',
  3,
  0),
 (41.52000045776367,
  116.0429916381836,
  154.27919006347656,
  128.0489959716797,
  'Aire géogr. — Syrie et Iraq.',
  4,
  0),
 (45.84000015258789,
  134.33700561523438,
  409.8203430175781,
  149.01101684570312,
  'HALOG

In [22]:
d = iter(doc)

In [23]:
next(d).getTextBlocks()

[(110.87999725341797,
  124.28401184082031,
  344.0527648925781,
  140.29200744628906,
  'AUTRES OUVRAGES DU MÊME AUTEUR',
  0,
  0),
 (59.040000915527344,
  183.02999877929688,
  419.97821044921875,
  196.3699951171875,
  'La végétation arborescente des Pays du Levant, dans : Publications techniques et',
  1,
  0),
 (35.52000045776367,
  195.89700317382812,
  370.8548889160156,
  210.57101440429688,
  "scientifiques de l'École Française d'Ingénieurs de Beyrouth, n° 13, 1947.",
  2,
  0),
 (59.279998779296875,
  218.07000732421875,
  420.3532409667969,
  231.41000366210938,
  "Contribution à l'étude de la flore syrienne et libanaise. Série de courts mémoires",
  3,
  0),
 (35.52000045776367,
  231.177001953125,
  408.53289794921875,
  245.85101318359375,
  'insérés dans : Bulletin de la Société Botanique de France, depuis 1952, tome 98.',
  4,
  0),
 (59.279998779296875,
  252.5369873046875,
  420.21917724609375,
  267.21099853515625,
  'La Flore du Djebel Druze. Université Saint-Josep

In [24]:
doc[621].getTextBlocks()

[(47.040000915527344,
  32.29714584350586,
  278.7561340332031,
  47.56802749633789,
  '546\nNOUVELLE FLORE',
  0,
  0),
 (58.79999923706055,
  59.93598556518555,
  143.0931396484375,
  290.4889831542969,
  'gussonii Pari.\nhordeaceus L.\nintermedius Guss.\njaponicus Thunb.\nlanceolatus Roth\nmacrpstachys Desf.\nmadritensis L.\nmaximus Desf.\nmollis L.\noxyphlaeus Payne\npalaestinus Melderis\npatulus Mert. et Koch\npinnatus L.\nramosus L.\nrigens L.\nrigidus Roth\nrubens L.\nscoparius L.\nsquarrosus L.\nsterilis L.\nsyriacus Boiss. et Bl.\ntectorum L.\ntomentellus Boiss.\nvillosus Forsk.',
  1,
  0),
 (45.119998931884766,
  287.843017578125,
  174.9159393310547,
  310.16900634765625,
  "Broussonetia papyrifera (L.) L'Hér.\nBuffonia",
  2,
  0),
 (58.31999969482422,
  307.0429992675781,
  130.1332550048828,
  376.8479919433594,
  'ephedrina Sam.\nkotschyana Boiss.\nmacrospermaj. Gay\nmulticeps Decne\npaniculata Dubois\ntenuifolia Vill.\nvirgata Boiss.',
  3,
  0),
 (44.400001525878906,


In [25]:
doc[621].getText()

"546\nNOUVELLE FLORE\ngussonii Pari.\nhordeaceus L.\nintermedius Guss.\njaponicus Thunb.\nlanceolatus Roth\nmacrpstachys Desf.\nmadritensis L.\nmaximus Desf.\nmollis L.\noxyphlaeus Payne\npalaestinus Melderis\npatulus Mert. et Koch\npinnatus L.\nramosus L.\nrigens L.\nrigidus Roth\nrubens L.\nscoparius L.\nsquarrosus L.\nsterilis L.\nsyriacus Boiss. et Bl.\ntectorum L.\ntomentellus Boiss.\nvillosus Forsk.\nBroussonetia papyrifera (L.) L'Hér.\nBuffonia\nephedrina Sam.\nkotschyana Boiss.\nmacrospermaj. Gay\nmulticeps Decne\npaniculata Dubois\ntenuifolia Vill.\nvirgata Boiss.\nBulbillaria gageoides Zucc.\nBUTOMACEAE\nButomus\numbellatus L.\nC\nCalamagrostis\nepigeios (L.) Roth\nlittorea (Schrad.) D.C.\npseudophragmites (Hall Fil.) Kœl.\nCalligonum\ncomosum L'Hér.\ntetrapterum Jaub. et Spach\nCannabis\nsativa L.\nCarex\nacuta L.\nacutiformis Ehrh.\nbanatica Heuff.\nbuekii Wimm.\ncontigua Hoppe\ndistans L.\ndiversicolor Druce\ndivisa Huds.\ndivulsa Stokes\n123\n129\n120, 128\n120, 129\n119,

In [26]:
page.getFontList()

[(48515, 'n/a', 'Type1', 'Times-Roman', 'F0', 'WinAnsiEncoding'),
 (48516, 'n/a', 'Type1', 'Times-Italic', 'F1', 'WinAnsiEncoding'),
 (48584, 'pfa', 'Type1', 'OYLUZG+Times-Roman', 'F12', 'WinAnsiEncoding'),
 (48575, 'pfa', 'Type1', 'RCBBCE+Times-Italic', 'F13', 'WinAnsiEncoding'),
 (48535, 'n/a', 'Type1', 'Times-Bold', 'F2', 'WinAnsiEncoding'),
 (48526, 'pfa', 'Type1', 'QKXTUK+Times-Roman', 'F24', 'WinAnsiEncoding'),
 (48517, 'pfa', 'Type1', 'KLTUIO+Times-Italic', 'F25', 'WinAnsiEncoding'),
 (48566, 'pfa', 'Type1', 'XTFERS+Times-Bold', 'F26', 'WinAnsiEncoding'),
 (48603, 'pfa', 'Type1', 'TCNEMC+Times-BoldItalic', 'F27', 'WinAnsiEncoding'),
 (48537, 'pfa', 'Type1', 'ETVWNV+Times-Roman', 'F28', 'WinAnsiEncoding'),
 (48546, 'pfa', 'Type1', 'CPDYZV+Times-Italic', 'F29', 'WinAnsiEncoding'),
 (48565, 'n/a', 'Type1', 'Times-BoldItalic', 'F3', 'WinAnsiEncoding'),
 (48556, 'pfa', 'Type1', 'VHVHMH+Times-Bold', 'F30', 'WinAnsiEncoding'),
 (48593, 'n/a', 'Type1', 'Helvetica', 'F4', 'WinAnsiEncodin

In [ ]:
import pandas as pd
df = pd.DataFrame(columns = columns)
df

In [ ]:
df = pd.read_excel('text_to_spreadsheet_template.xlsx')